In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, Input, Conv1D, Lambda, Flatten
from tensorflow.keras.losses import CategoricalFocalCrossentropy

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [4]:
def create_convnext_like_model(input_dim, num_classes, sliding_window_size=3):
    # Input layer
    input_layer = Input(shape=(input_dim,))
    
    # Reshape input to be compatible with Conv1D using Lambda layer
    reshaped_input = Lambda(lambda x: tf.expand_dims(x, axis=-1))(input_layer)  # Shape: (batch_size, input_dim, 1)
    x = reshaped_input
    
    # Simulate ConvNeXt block using Conv1D layers
    x = Conv1D(filters=int(max(16, input_dim * 2)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 2)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 4)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 8)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    
    # Flatten the output of the final Conv1D layer
    x = Flatten()(x)
    
    # Global pooling and classification head
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_focal_crossentropy', metrics=['accuracy'])
    
    return model

In [5]:
def run_model(train_data_path, test_data_path, is_string_labels = False, label_mapping = None):

    # Initialize the one-hot encoder for the target
    encoder = OneHotEncoder(sparse_output=False)

    # Load and Prepare Training Data
    train_data = pd.read_csv(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        train_data['label'] = train_data['label'].map(label_mapping)
    train_X = train_data.drop(columns=['label']).values
    train_y = train_data['label'].values
    train_y = encoder.fit_transform(train_y.reshape(-1, 1))
    del train_data
    gc.collect()

    # Load and Prepare Test Data (this will not be used in training)
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        test_data['label'] = test_data['label'].map(label_mapping)
    test_X = test_data.drop(columns=['label']).values
    test_y = test_data['label'].values
    test_y = encoder.transform(test_y.reshape(-1, 1))
    del test_data
    gc.collect()

    # EarlyStopping Callback (optional, to avoid overfitting)
    early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

    # Number of runs for averaging results
    num_runs = 50

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    # Train the Model with Validation Split N tines for more accurate metrics
    #print("Verbose output only for first run...")
    verbose_run = 0
    for run in range(num_runs):
        
        # Model is defined separately in each run, since the random combination layers
        # must be randomly initialized each time. Otherwise, the "random" indices stay the same
        # throughout all runs
        model = create_convnext_like_model(train_X.shape[1], test_y.shape[1])

        print(f"Run {run + 1}/{num_runs} started...")
        history = model.fit(
            train_X, train_y, 
            epochs=100, 
            batch_size=int(train_X.shape[0] * 0.01),
            callbacks=[early_stopping],
            verbose=verbose_run
        )
        verbose_run = 0 # Suppress detailed output for multiple runs

        test_loss, test_acc = model.evaluate(test_X, test_y, verbose=0)
        y_pred = model.predict(test_X, verbose=0)
        y_pred_classes = y_pred.argmax(axis=1)
        y_true_classes = test_y.argmax(axis=1)
        del model
        gc.collect()

        # Compute metrics
        balanced_acc = balanced_accuracy_score(y_true_classes, y_pred_classes)
        roc_auc = roc_auc_score(test_y, y_pred, multi_class='ovr')  # `test_y` is fine here for AUC
        pr_auc = average_precision_score(test_y, y_pred, average='weighted')
        f2 = fbeta_score(y_true_classes, y_pred_classes, beta=2, average='weighted')
        mcc = matthews_corrcoef(y_true_classes, y_pred_classes)

        # Store metrics
        metrics_storage['test_loss'].append(test_loss)
        metrics_storage['test_accuracy'].append(test_acc)
        metrics_storage['balanced_accuracy'].append(balanced_acc)
        metrics_storage['roc_auc'].append(roc_auc)
        metrics_storage['pr_auc'].append(pr_auc)
        metrics_storage['f2'].append(f2)
        metrics_storage['mcc'].append(mcc)

        # Store classification report metrics
        report = classification_report(y_true_classes, y_pred_classes, output_dict=True)
        for label, values in report.items():
            # Check if the value is a dictionary (e.g., 'precision', 'recall', 'f1-score')
            if isinstance(values, dict):
                for metric, value in values.items():
                    metrics_storage[f"{label}_{metric}"].append(value)
            else:
                # Handle scalar values (like 'accuracy')
                metrics_storage[label].append(values)

        # Average the metrics over all successful runs
        print(f"\nAggregated Metrics for {run+1} runs:")
        for metric, values in metrics_storage.items():
            avg_value = np.mean(values)
            print(f"{metric}: {avg_value:.4f}")

    gc.collect()
        

In [ ]:
run_model("/kaggle/input/ma-datasets/shuttle_train.csv", "/kaggle/input/ma-datasets/shuttle_test.csv", is_string_labels = False)

Run 1/50 started...

Aggregated Metrics for 1 runs:
test_loss: 0.0003
test_accuracy: 0.9992
balanced_accuracy: 0.9112
roc_auc: 1.0000
pr_auc: 0.9999
f2: 0.9992
mcc: 0.9978
0_precision: 0.9995
0_recall: 1.0000
0_f1-score: 0.9997
0_support: 9117.0000
1_precision: 1.0000
1_recall: 1.0000
1_f1-score: 1.0000
1_support: 10.0000
2_precision: 0.9091
2_recall: 0.8824
2_f1-score: 0.8955
2_support: 34.0000
3_precision: 1.0000
3_recall: 0.9989
3_f1-score: 0.9994
3_support: 1781.0000
4_precision: 0.9985
4_recall: 0.9969
4_f1-score: 0.9977
4_support: 653.0000
5_precision: 1.0000
5_recall: 0.5000
5_f1-score: 0.6667
5_support: 2.0000
6_precision: 1.0000
6_recall: 1.0000
6_f1-score: 1.0000
6_support: 3.0000
accuracy: 0.9992
macro avg_precision: 0.9867
macro avg_recall: 0.9112
macro avg_f1-score: 0.9370
macro avg_support: 11600.0000
weighted avg_precision: 0.9992
weighted avg_recall: 0.9992
weighted avg_f1-score: 0.9992
weighted avg_support: 11600.0000
Run 2/50 started...

Aggregated Metrics for 2 runs:

In [ ]:
run_model("/kaggle/input/ma-datasets/covtype_train.csv", "/kaggle/input/ma-datasets/covtype_test.csv", is_string_labels = False)

Run 1/50 started...



Aggregated Metrics for 1 runs:
test_loss: 0.0233
test_accuracy: 0.8719
balanced_accuracy: 0.7558
roc_auc: 0.9872
pr_auc: 0.9467
f2: 0.8708
mcc: 0.7923
0_precision: 0.8938
0_recall: 0.8471
0_f1-score: 0.8698
0_support: 42368.0000
1_precision: 0.8623
1_recall: 0.9250
1_f1-score: 0.8925
1_support: 56661.0000
2_precision: 0.8540
2_recall: 0.8344
2_f1-score: 0.8441
2_support: 7151.0000
3_precision: 0.8606
3_recall: 0.6521
3_f1-score: 0.7420
3_support: 549.0000
4_precision: 0.7837
4_recall: 0.5914
4_f1-score: 0.6741
4_support: 1899.0000
5_precision: 0.7730
5_recall: 0.5462
5_f1-score: 0.6401
5_support: 3473.0000
6_precision: 0.9226
6_recall: 0.8947
6_f1-score: 0.9084
6_support: 4102.0000
accuracy: 0.8719
macro avg_precision: 0.8500
macro avg_recall: 0.7558
macro avg_f1-score: 0.7959
macro avg_support: 116203.0000
weighted avg_precision: 0.8714
weighted avg_recall: 0.8719
weighted avg_f1-score: 0.8700
weighted avg_support: 116203.0000
Run 2/50 started...



Aggregated Metrics for 2 runs:
test_loss: 0.0405
test_accuracy: 0.7988
balanced_accuracy: 0.6331
roc_auc: 0.9649
pr_auc: 0.8700
f2: 0.7956
mcc: 0.6778
0_precision: 0.7882
0_recall: 0.8187
0_f1-score: 0.8012
0_support: 42368.0000
1_precision: 0.8269
1_recall: 0.8210
1_f1-score: 0.8225
1_support: 56661.0000
2_precision: 0.7491
2_recall: 0.8520
2_f1-score: 0.7921
2_support: 7151.0000
3_precision: 0.7467
3_recall: 0.3998
3_f1-score: 0.4906
3_support: 549.0000
4_precision: 0.7736
4_recall: 0.3628
4_f1-score: 0.4512
4_support: 1899.0000
5_precision: 0.6438
5_recall: 0.3395
5_f1-score: 0.4256
5_support: 3473.0000
6_precision: 0.7944
6_recall: 0.8375
6_f1-score: 0.8136
6_support: 4102.0000
accuracy: 0.7988
macro avg_precision: 0.7604
macro avg_recall: 0.6331
macro avg_f1-score: 0.6567
macro avg_support: 116203.0000
weighted avg_precision: 0.8001
weighted avg_recall: 0.7988
weighted avg_f1-score: 0.7930
weighted avg_support: 116203.0000
Run 3/50 started...



Aggregated Metrics for 3 runs:
test_loss: 0.0472
test_accuracy: 0.7709
balanced_accuracy: 0.5725
roc_auc: 0.9563
pr_auc: 0.8417
f2: 0.7660
mcc: 0.6339
0_precision: 0.7438
0_recall: 0.8247
0_f1-score: 0.7790
0_support: 42368.0000
1_precision: 0.8155
1_recall: 0.7770
1_f1-score: 0.7940
1_support: 56661.0000
2_precision: 0.7125
2_recall: 0.8604
2_f1-score: 0.7746
2_support: 7151.0000
3_precision: 0.7129
3_recall: 0.3097
3_f1-score: 0.3989
3_support: 549.0000
4_precision: 0.7449
4_recall: 0.2824
4_f1-score: 0.3697
4_support: 1899.0000
5_precision: 0.6063
5_recall: 0.2361
5_f1-score: 0.3023
5_support: 3473.0000
6_precision: 0.8031
6_recall: 0.7170
6_f1-score: 0.7432
6_support: 4102.0000
accuracy: 0.7709
macro avg_precision: 0.7341
macro avg_recall: 0.5725
macro avg_f1-score: 0.5945
macro avg_support: 116203.0000
weighted avg_precision: 0.7747
weighted avg_recall: 0.7709
weighted avg_f1-score: 0.7621
weighted avg_support: 116203.0000
Run 4/50 started...



Aggregated Metrics for 4 runs:
test_loss: 0.0501
test_accuracy: 0.7616
balanced_accuracy: 0.5458
roc_auc: 0.9525
pr_auc: 0.8286
f2: 0.7563
mcc: 0.6169
0_precision: 0.7380
0_recall: 0.8033
0_f1-score: 0.7667
0_support: 42368.0000
1_precision: 0.8028
1_recall: 0.7797
1_f1-score: 0.7895
1_support: 56661.0000
2_precision: 0.6864
2_recall: 0.8723
2_f1-score: 0.7631
2_support: 7151.0000
3_precision: 0.7178
3_recall: 0.2559
3_f1-score: 0.3411
3_support: 549.0000
4_precision: 0.7271
4_recall: 0.2328
4_f1-score: 0.3145
4_support: 1899.0000
5_precision: 0.5763
5_recall: 0.2025
5_f1-score: 0.2687
5_support: 3473.0000
6_precision: 0.8112
6_recall: 0.6744
6_f1-score: 0.7227
6_support: 4102.0000
accuracy: 0.7616
macro avg_precision: 0.7228
macro avg_recall: 0.5458
macro avg_f1-score: 0.5666
macro avg_support: 116203.0000
weighted avg_precision: 0.7639
weighted avg_recall: 0.7616
weighted avg_f1-score: 0.7518
weighted avg_support: 116203.0000
Run 5/50 started...


...


Aggregated Metrics for 10 runs:
test_loss: 0.0555
test_accuracy: 0.7412
balanced_accuracy: 0.4978
roc_auc: 0.9454
pr_auc: 0.8031
f2: 0.7352
mcc: 0.5800
0_precision: 0.7269
0_recall: 0.7564
0_f1-score: 0.7380
0_support: 42368.0000
1_precision: 0.7747
1_recall: 0.7898
1_f1-score: 0.7803
1_support: 56661.0000
2_precision: 0.6673
2_recall: 0.8309
2_f1-score: 0.7339
2_support: 7151.0000
3_precision: 0.6775
3_recall: 0.1989
3_f1-score: 0.2807
3_support: 549.0000
4_precision: 0.6459
4_recall: 0.1412
4_f1-score: 0.2086
4_support: 1899.0000
5_precision: 0.5214
5_recall: 0.2093
5_f1-score: 0.2601
5_support: 3473.0000
6_precision: 0.8238
6_recall: 0.5577
6_f1-score: 0.6350
6_support: 4102.0000
accuracy: 0.7412
macro avg_precision: 0.6911
macro avg_recall: 0.4978
macro avg_f1-score: 0.5195
macro avg_support: 116203.0000
weighted avg_precision: 0.7422
weighted avg_recall: 0.7412
weighted avg_f1-score: 0.7297
weighted avg_support: 116203.0000
Run 11/50 started...



Aggregated Metrics for 11 runs:
test_loss: 0.0557
test_accuracy: 0.7412
balanced_accuracy: 0.4969
roc_auc: 0.9451
pr_auc: 0.8024
f2: 0.7353
mcc: 0.5798
0_precision: 0.7283
0_recall: 0.7522
0_f1-score: 0.7370
0_support: 42368.0000
1_precision: 0.7734
1_recall: 0.7918
1_f1-score: 0.7808
1_support: 56661.0000
2_precision: 0.6657
2_recall: 0.8335
2_f1-score: 0.7345
2_support: 7151.0000
3_precision: 0.6784
3_recall: 0.1845
3_f1-score: 0.2620
3_support: 549.0000
4_precision: 0.6646
4_recall: 0.1347
4_f1-score: 0.2013
4_support: 1899.0000
5_precision: 0.5194
5_recall: 0.2113
5_f1-score: 0.2652
5_support: 3473.0000
6_precision: 0.8148
6_recall: 0.5703
6_f1-score: 0.6419
6_support: 4102.0000
accuracy: 0.7412
macro avg_precision: 0.6921
macro avg_recall: 0.4969
macro avg_f1-score: 0.5175
macro avg_support: 116203.0000
weighted avg_precision: 0.7420
weighted avg_recall: 0.7412
weighted avg_f1-score: 0.7297
weighted avg_support: 116203.0000
Run 12/50 started...



Aggregated Metrics for 12 runs:
test_loss: 0.0559
test_accuracy: 0.7407
balanced_accuracy: 0.4985
roc_auc: 0.9449
pr_auc: 0.8012
f2: 0.7348
mcc: 0.5792
0_precision: 0.7297
0_recall: 0.7474
0_f1-score: 0.7354
0_support: 42368.0000
1_precision: 0.7726
1_recall: 0.7925
1_f1-score: 0.7808
1_support: 56661.0000
2_precision: 0.6610
2_recall: 0.8400
2_f1-score: 0.7341
2_support: 7151.0000
3_precision: 0.6917
3_recall: 0.1793
3_f1-score: 0.2579
3_support: 549.0000
4_precision: 0.6808
4_recall: 0.1328
4_f1-score: 0.2010
4_support: 1899.0000
5_precision: 0.5181
5_recall: 0.2093
5_f1-score: 0.2659
5_support: 3473.0000
6_precision: 0.8002
6_recall: 0.5880
6_f1-score: 0.6470
6_support: 4102.0000
accuracy: 0.7407
macro avg_precision: 0.6934
macro avg_recall: 0.4985
macro avg_f1-score: 0.5175
macro avg_support: 116203.0000
weighted avg_precision: 0.7416
weighted avg_recall: 0.7407
weighted avg_f1-score: 0.7293
weighted avg_support: 116203.0000
Run 13/50 started...



Aggregated Metrics for 13 runs:
test_loss: 0.0558
test_accuracy: 0.7407
balanced_accuracy: 0.5000
roc_auc: 0.9449
pr_auc: 0.8012
f2: 0.7349
mcc: 0.5791
0_precision: 0.7309
0_recall: 0.7445
0_f1-score: 0.7348
0_support: 42368.0000
1_precision: 0.7713
1_recall: 0.7946
1_f1-score: 0.7813
1_support: 56661.0000
2_precision: 0.6594
2_recall: 0.8413
2_f1-score: 0.7341
2_support: 7151.0000
3_precision: 0.6772
3_recall: 0.1920
3_f1-score: 0.2696
3_support: 549.0000
4_precision: 0.6777
4_recall: 0.1332
4_f1-score: 0.2030
4_support: 1899.0000
5_precision: 0.5210
5_recall: 0.2014
5_f1-score: 0.2591
5_support: 3473.0000
6_precision: 0.7992
6_recall: 0.5931
6_f1-score: 0.6522
6_support: 4102.0000
accuracy: 0.7407
macro avg_precision: 0.6910
macro avg_recall: 0.5000
macro avg_f1-score: 0.5192
macro avg_support: 116203.0000
weighted avg_precision: 0.7412
weighted avg_recall: 0.7407
weighted avg_f1-score: 0.7294
weighted avg_support: 116203.0000
Run 14/50 started...



Aggregated Metrics for 14 runs:
test_loss: 0.0560
test_accuracy: 0.7401
balanced_accuracy: 0.4987
roc_auc: 0.9445
pr_auc: 0.8003
f2: 0.7343
mcc: 0.5776
0_precision: 0.7307
0_recall: 0.7420
0_f1-score: 0.7337
0_support: 42368.0000
1_precision: 0.7696
1_recall: 0.7960
1_f1-score: 0.7811
1_support: 56661.0000
2_precision: 0.6585
2_recall: 0.8414
2_f1-score: 0.7339
2_support: 7151.0000
3_precision: 0.6733
3_recall: 0.1995
3_f1-score: 0.2790
3_support: 549.0000
4_precision: 0.6788
4_recall: 0.1291
4_f1-score: 0.1983
4_support: 1899.0000
5_precision: 0.5235
5_recall: 0.1948
5_f1-score: 0.2536
5_support: 3473.0000
6_precision: 0.8008
6_recall: 0.5879
6_f1-score: 0.6511
6_support: 4102.0000
accuracy: 0.7401
macro avg_precision: 0.6907
macro avg_recall: 0.4987
macro avg_f1-score: 0.5187
macro avg_support: 116203.0000
weighted avg_precision: 0.7404
weighted avg_recall: 0.7401
weighted avg_f1-score: 0.7287
weighted avg_support: 116203.0000
Run 15/50 started...


In [ ]:
labels_map = {
    'normal.': 0, 'satan.': 1, 'ipsweep.': 2, 'portsweep.': 3, 'nmap.': 4,
    'back.': 5, 'warezclient.': 6, 'teardrop.': 7, 'pod.': 8, 'guess_passwd.': 9,
    'buffer_overflow.': 10, 'land.': 11, 'warezmaster.': 12, 'imap.': 13, 'rootkit.': 14,
    'loadmodule.': 15, 'multihop.': 16, 'ftp_write.': 17, 'phf.': 18, 'perl.': 19, 'spy.': 20
}

run_model("kdd_train.csv", "kdd_test.csv", is_string_labels = True, label_mapping=labels_map)


Run 1/50 started...

Aggregated Metrics for 1 runs:
test_loss: 0.0002
test_accuracy: 0.9994
balanced_accuracy: 0.8144
roc_auc: 0.9999
pr_auc: 0.9999
f2: 0.9994
mcc: 0.9927
0_precision: 0.9996
0_recall: 0.9999
0_f1-score: 0.9998
0_support: 194557.0000
1_precision: 0.9997
1_recall: 0.9953
1_f1-score: 0.9975
1_support: 3178.0000
2_precision: 0.9955
2_recall: 0.9768
2_f1-score: 0.9860
2_support: 2496.0000
3_precision: 1.0000
3_recall: 0.9966
3_f1-score: 0.9983
3_support: 2083.0000
4_precision: 0.9204
4_recall: 0.9741
4_f1-score: 0.9465
4_support: 463.0000
5_precision: 1.0000
5_recall: 1.0000
5_f1-score: 1.0000
5_support: 441.0000
6_precision: 0.9689
6_recall: 0.9167
6_f1-score: 0.9421
6_support: 204.0000
7_precision: 1.0000
7_recall: 1.0000
7_f1-score: 1.0000
7_support: 196.0000
8_precision: 1.0000
8_recall: 1.0000
8_f1-score: 1.0000
8_support: 53.0000
9_precision: 1.0000
9_recall: 0.9091
9_f1-score: 0.9524
9_support: 11.0000
10_precision: 1.0000
10_recall: 0.8333
10_f1-score: 0.9091
10_s

In [ ]:
labels_map = {
    'Normal': 0, 'Darknet_Audio-Streaming': 1, 'Darknet_Chat': 2, 'Darknet_File-Transfer': 3, 'Darknet_VOIP': 4,
    'Darknet_Video-Streaming': 5, 'Darknet_Email': 6, 'Darknet_Browsing': 7, 'Darknet_P2P': 8
}

run_model("/kaggle/input/ma-datasets/darknet_train.csv", "/kaggle/input/ma-datasets/darknet_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/50 started...



Aggregated Metrics for 1 runs:
test_loss: 0.0004
test_accuracy: 0.9991
balanced_accuracy: 0.9791
roc_auc: 0.9999
pr_auc: 0.9999
f2: 0.9991
mcc: 0.9969
0_precision: 0.9997
0_recall: 0.9999
0_f1-score: 0.9998
0_support: 26862.0000
1_precision: 0.9992
1_recall: 0.9985
1_f1-score: 0.9989
1_support: 2657.0000
2_precision: 0.9956
2_recall: 0.9956
2_f1-score: 0.9956
2_support: 908.0000
3_precision: 0.9962
3_recall: 0.9981
3_f1-score: 0.9971
3_support: 522.0000
4_precision: 0.9764
4_recall: 0.9898
4_f1-score: 0.9831
4_support: 293.0000
5_precision: 0.9925
5_recall: 0.9814
5_f1-score: 0.9869
5_support: 269.0000
6_precision: 0.9915
6_recall: 1.0000
6_f1-score: 0.9957
6_support: 116.0000
7_precision: 0.9574
7_recall: 0.8491
7_f1-score: 0.9000
7_support: 53.0000
8_precision: 1.0000
8_recall: 1.0000
8_f1-score: 1.0000
8_support: 44.0000
accuracy: 0.9991
macro avg_precision: 0.9898
macro avg_recall: 0.9791
macro avg_f1-score: 0.9841
macro avg_support: 31724.0000
weighted avg_precision: 0.9991
weigh


Aggregated Metrics for 2 runs:
test_loss: 0.0028
test_accuracy: 0.9944
balanced_accuracy: 0.9266
roc_auc: 0.9993
pr_auc: 0.9980
f2: 0.9944
mcc: 0.9797
0_precision: 0.9981
0_recall: 0.9991
0_f1-score: 0.9986
0_support: 26862.0000
1_precision: 0.9907
1_recall: 0.9883
1_f1-score: 0.9895
1_support: 2657.0000
2_precision: 0.9840
2_recall: 0.9813
2_f1-score: 0.9826
2_support: 908.0000
3_precision: 0.9380
3_recall: 0.9617
3_f1-score: 0.9495
3_support: 522.0000
4_precision: 0.9343
4_recall: 0.9471
4_f1-score: 0.9407
4_support: 293.0000
5_precision: 0.9178
5_recall: 0.8401
5_f1-score: 0.8756
5_support: 269.0000
6_precision: 0.9666
6_recall: 0.9181
6_f1-score: 0.9408
6_support: 116.0000
7_precision: 0.8372
7_recall: 0.7830
7_f1-score: 0.8085
7_support: 53.0000
8_precision: 0.9625
8_recall: 0.9205
8_f1-score: 0.9405
8_support: 44.0000
accuracy: 0.9944
macro avg_precision: 0.9477
macro avg_recall: 0.9266
macro avg_f1-score: 0.9362
macro avg_support: 31724.0000
weighted avg_precision: 0.9944
weigh


Aggregated Metrics for 3 runs:
test_loss: 0.0034
test_accuracy: 0.9931
balanced_accuracy: 0.9022
roc_auc: 0.9988
pr_auc: 0.9975
f2: 0.9930
mcc: 0.9748
0_precision: 0.9975
0_recall: 0.9989
0_f1-score: 0.9982
0_support: 26862.0000
1_precision: 0.9876
1_recall: 0.9841
1_f1-score: 0.9859
1_support: 2657.0000
2_precision: 0.9784
2_recall: 0.9809
2_f1-score: 0.9797
2_support: 908.0000
3_precision: 0.9185
3_recall: 0.9585
3_f1-score: 0.9378
3_support: 522.0000
4_precision: 0.9510
4_recall: 0.9181
4_f1-score: 0.9331
4_support: 293.0000
5_precision: 0.8980
5_recall: 0.8079
5_f1-score: 0.8493
5_support: 269.0000
6_precision: 0.9533
6_recall: 0.9023
6_f1-score: 0.9264
6_support: 116.0000
7_precision: 0.8045
7_recall: 0.7358
7_f1-score: 0.7679
7_support: 53.0000
8_precision: 0.9639
8_recall: 0.8333
8_f1-score: 0.8882
8_support: 44.0000
accuracy: 0.9931
macro avg_precision: 0.9392
macro avg_recall: 0.9022
macro avg_f1-score: 0.9185
macro avg_support: 31724.0000
weighted avg_precision: 0.9930
weigh


Aggregated Metrics for 4 runs:
test_loss: 0.0040
test_accuracy: 0.9919
balanced_accuracy: 0.8872
roc_auc: 0.9985
pr_auc: 0.9971
f2: 0.9918
mcc: 0.9703
0_precision: 0.9963
0_recall: 0.9991
0_f1-score: 0.9977
0_support: 26862.0000
1_precision: 0.9895
1_recall: 0.9783
1_f1-score: 0.9838
1_support: 2657.0000
2_precision: 0.9779
2_recall: 0.9755
2_f1-score: 0.9767
2_support: 908.0000
3_precision: 0.9127
3_recall: 0.9478
3_f1-score: 0.9297
3_support: 522.0000
4_precision: 0.9582
4_recall: 0.8951
4_f1-score: 0.9239
4_support: 293.0000
5_precision: 0.8735
5_recall: 0.7918
5_f1-score: 0.8297
5_support: 269.0000
6_precision: 0.9537
6_recall: 0.8599
6_f1-score: 0.9021
6_support: 116.0000
7_precision: 0.7753
7_recall: 0.7075
7_f1-score: 0.7393
7_support: 53.0000
8_precision: 0.9229
8_recall: 0.8295
8_f1-score: 0.8684
8_support: 44.0000
accuracy: 0.9919
macro avg_precision: 0.9289
macro avg_recall: 0.8872
macro avg_f1-score: 0.9057
macro avg_support: 31724.0000
weighted avg_precision: 0.9918
weigh

...


Aggregated Metrics for 8 runs:
test_loss: 0.0048
test_accuracy: 0.9899
balanced_accuracy: 0.8682
roc_auc: 0.9981
pr_auc: 0.9965
f2: 0.9897
mcc: 0.9627
0_precision: 0.9945
0_recall: 0.9992
0_f1-score: 0.9969
0_support: 26862.0000
1_precision: 0.9892
1_recall: 0.9725
1_f1-score: 0.9807
1_support: 2657.0000
2_precision: 0.9761
2_recall: 0.9628
2_f1-score: 0.9694
2_support: 908.0000
3_precision: 0.9126
3_recall: 0.9176
3_f1-score: 0.9138
3_support: 522.0000
4_precision: 0.9619
4_recall: 0.8281
4_f1-score: 0.8855
4_support: 293.0000
5_precision: 0.8283
5_recall: 0.7876
5_f1-score: 0.8062
5_support: 269.0000
6_precision: 0.9488
6_recall: 0.8394
6_f1-score: 0.8891
6_support: 116.0000
7_precision: 0.7356
7_recall: 0.6745
7_f1-score: 0.7013
7_support: 53.0000
8_precision: 0.9138
8_recall: 0.8324
8_f1-score: 0.8679
8_support: 44.0000
accuracy: 0.9899
macro avg_precision: 0.9179
macro avg_recall: 0.8682
macro avg_f1-score: 0.8901
macro avg_support: 31724.0000
weighted avg_precision: 0.9898
weigh


Aggregated Metrics for 9 runs:
test_loss: 0.0048
test_accuracy: 0.9899
balanced_accuracy: 0.8680
roc_auc: 0.9982
pr_auc: 0.9965
f2: 0.9898
mcc: 0.9628
0_precision: 0.9947
0_recall: 0.9992
0_f1-score: 0.9969
0_support: 26862.0000
1_precision: 0.9883
1_recall: 0.9724
1_f1-score: 0.9802
1_support: 2657.0000
2_precision: 0.9764
2_recall: 0.9626
2_f1-score: 0.9694
2_support: 908.0000
3_precision: 0.9156
3_recall: 0.9161
3_f1-score: 0.9147
3_support: 522.0000
4_precision: 0.9644
4_recall: 0.8305
4_f1-score: 0.8884
4_support: 293.0000
5_precision: 0.8159
5_recall: 0.7955
5_f1-score: 0.8034
5_support: 269.0000
6_precision: 0.9419
6_recall: 0.8439
6_f1-score: 0.8884
6_support: 116.0000
7_precision: 0.7379
7_recall: 0.6583
7_f1-score: 0.6925
7_support: 53.0000
8_precision: 0.9125
8_recall: 0.8333
8_f1-score: 0.8682
8_support: 44.0000
accuracy: 0.9899
macro avg_precision: 0.9164
macro avg_recall: 0.8680
macro avg_f1-score: 0.8891
macro avg_support: 31724.0000
weighted avg_precision: 0.9898
weigh


Aggregated Metrics for 10 runs:
test_loss: 0.0048
test_accuracy: 0.9898
balanced_accuracy: 0.8679
roc_auc: 0.9982
pr_auc: 0.9965
f2: 0.9896
mcc: 0.9624
0_precision: 0.9946
0_recall: 0.9992
0_f1-score: 0.9969
0_support: 26862.0000
1_precision: 0.9869
1_recall: 0.9723
1_f1-score: 0.9795
1_support: 2657.0000
2_precision: 0.9765
2_recall: 0.9608
2_f1-score: 0.9685
2_support: 908.0000
3_precision: 0.9185
3_recall: 0.9149
3_f1-score: 0.9156
3_support: 522.0000
4_precision: 0.9652
4_recall: 0.8311
4_f1-score: 0.8895
4_support: 293.0000
5_precision: 0.8176
5_recall: 0.7844
5_f1-score: 0.7982
5_support: 269.0000
6_precision: 0.9389
6_recall: 0.8483
6_f1-score: 0.8896
6_support: 116.0000
7_precision: 0.7297
7_recall: 0.6679
7_f1-score: 0.6934
7_support: 53.0000
8_precision: 0.9160
8_recall: 0.8318
8_f1-score: 0.8692
8_support: 44.0000
accuracy: 0.9898
macro avg_precision: 0.9160
macro avg_recall: 0.8679
macro avg_f1-score: 0.8889
macro avg_support: 31724.0000
weighted avg_precision: 0.9897
weig


Aggregated Metrics for 11 runs:
test_loss: 0.0049
test_accuracy: 0.9894
balanced_accuracy: 0.8626
roc_auc: 0.9981
pr_auc: 0.9964
f2: 0.9892
mcc: 0.9608
0_precision: 0.9942
0_recall: 0.9992
0_f1-score: 0.9967
0_support: 26862.0000
1_precision: 0.9874
1_recall: 0.9722
1_f1-score: 0.9797
1_support: 2657.0000
2_precision: 0.9767
2_recall: 0.9615
2_f1-score: 0.9690
2_support: 908.0000
3_precision: 0.9193
3_recall: 0.9121
3_f1-score: 0.9146
3_support: 522.0000
4_precision: 0.9591
4_recall: 0.7965
4_f1-score: 0.8632
4_support: 293.0000
5_precision: 0.8129
5_recall: 0.7803
5_f1-score: 0.7940
5_support: 269.0000
6_precision: 0.9250
6_recall: 0.8519
6_f1-score: 0.8845
6_support: 116.0000
7_precision: 0.7327
7_recall: 0.6569
7_f1-score: 0.6883
7_support: 53.0000
8_precision: 0.9168
8_recall: 0.8326
8_f1-score: 0.8703
8_support: 44.0000
accuracy: 0.9894
macro avg_precision: 0.9138
macro avg_recall: 0.8626
macro avg_f1-score: 0.8845
macro avg_support: 31724.0000
weighted avg_precision: 0.9892
weig


Aggregated Metrics for 12 runs:
test_loss: 0.0050
test_accuracy: 0.9892
balanced_accuracy: 0.8612
roc_auc: 0.9981
pr_auc: 0.9963
f2: 0.9890
mcc: 0.9601
0_precision: 0.9940
0_recall: 0.9992
0_f1-score: 0.9966
0_support: 26862.0000
1_precision: 0.9875
1_recall: 0.9714
1_f1-score: 0.9794
1_support: 2657.0000
2_precision: 0.9761
2_recall: 0.9596
2_f1-score: 0.9677
2_support: 908.0000
3_precision: 0.9183
3_recall: 0.9095
3_f1-score: 0.9129
3_support: 522.0000
4_precision: 0.9581
4_recall: 0.8018
4_f1-score: 0.8664
4_support: 293.0000
5_precision: 0.8126
5_recall: 0.7720
5_f1-score: 0.7895
5_support: 269.0000
6_precision: 0.9259
6_recall: 0.8534
6_f1-score: 0.8859
6_support: 116.0000
7_precision: 0.7367
7_recall: 0.6525
7_f1-score: 0.6877
7_support: 53.0000
8_precision: 0.9194
8_recall: 0.8314
8_f1-score: 0.8709
8_support: 44.0000
accuracy: 0.9892
macro avg_precision: 0.9143
macro avg_recall: 0.8612
macro avg_f1-score: 0.8841
macro avg_support: 31724.0000
weighted avg_precision: 0.9890
weig